<a id='0'></a>
# Part 1. Problem Definition

This problem is to design an automated trading solution for single stock trading. We model the stock trading process as a Markov Decision Process (MDP). We then formulate our trading goal as a maximization problem.

The algorithm is trained using Deep Reinforcement Learning (DRL) algorithms and the components of the reinforcement learning environment are:


* Action: 
* Reward function: 
* State: 
* Environment: 

<a id='1'></a>
# Part 2. Getting Started- Load Python Packages

<a id='1.1'></a>
## 2.1. Install all the packages

In [7]:
!pip install -r requirements.txt --user

<a id='1.2'></a>
## 2.2. Import Packages

In [8]:
from config import config
from dataset.download_dataset.cryptodownloader import CryptoDownloader

<a id='1.3'></a>
## 2.3 Create Folders

In [9]:
import os
download_data = False
if not os.path.exists("./" + config.DATA_SAVE_DIR):
    os.makedirs("./" + config.DATA_SAVE_DIR)
    download_data = True
    
# if not os.path.exists("./" + config.TRAINED_MODEL_DIR):
#     os.makedirs("./" + config.TRAINED_MODEL_DIR)
# if not os.path.exists("./" + config.TENSORBOARD_LOG_DIR):
#     os.makedirs("./" + config.TENSORBOARD_LOG_DIR)
# if not os.path.exists("./" + config.RESULTS_DIR):
#     os.makedirs("./" + config.RESULTS_DIR)

<a id='2'></a>
# Part 3. Download Data

In [10]:
data_downloader = CryptoDownloader('2021-05-01', '2021-05-02', ['BTC', 'ETH'])
if download_data:    
    data_downloader.download_data()
df = data_downloader.load()

In [11]:
df.head()

,open,close,high,low,volume,ticker,datetime
3319486,57477.380115,57469.000000,57489.000000,57350.000,11.248819,btc,2021-05-01 00:01:00
3319487,57471.852000,57629.922062,57629.922062,57471.852,6.466445,btc,2021-05-01 00:02:00
3319488,57629.922062,57653.000000,57668.000000,57629.000,3.098331,btc,2021-05-01 00:03:00
3319489,57650.130000,57692.000000,57697.118000,57650.130,6.221445,btc,2021-05-01 00:04:00
3319490,57691.000000,57737.796088,57765.000000,57691.000,3.184753,btc,2021-05-01 00:05:00


In [12]:
df.shape

(2875, 7)

# Part 4: Preprocess Data

<a id='4'></a>
# Part 5. Design Environment
Considering the stochastic and interactive nature of the automated stock trading tasks, a financial task is modeled as a **Markov Decision Process (MDP)** problem. The training process involves observing stock price change, taking an action and reward's calculation to have the agent adjusting its strategy accordingly. By interacting with the environment, the trading agent will derive a trading strategy with the maximized rewards as time proceeds.

Our trading environments, based on OpenAI Gym framework, simulate live stock markets with real market data according to the principle of time-driven simulation.

The action space describes the allowed actions that the agent interacts with the environment. Normally, action a includes three actions: {-1, 0, 1}, where -1, 0, 1 represent selling, holding, and buying one share. Also, an action can be carried upon multiple shares. We use an action space {-k,…,-1, 0, 1, …, k}, where k denotes the number of shares to buy and -k denotes the number of shares to sell. For example, "Buy 10 shares of AAPL" or "Sell 10 shares of AAPL" are 10 or -10, respectively. The continuous action space needs to be normalized to [-1, 1], since the policy is defined on a Gaussian distribution, which needs to be normalized and symmetric.

<a id='5'></a>
# Part 6: Implement DRL Algorithms

<a id='6'></a>
# Part 7: Backtest